### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several reasons in favor of choosing AutoGen for your AI Agent project:

1. **Self-Improvement**: AutoGen features self-improving agent collaboration, meaning that agents can learn from experiences and improve their performance over time.

2. **Dynamic Role-Playing**: It supports dynamic role assignments, allowing AI agents to switch roles based on task requirements. This flexibility can enhance problem-solving abilities and optimize workflows.

3. **Efficiency**: By enabling agents to reassign tasks and iterate on solutions without constant human intervention, AutoGen can lead to more efficient processes and reduced operational overhead.

4. **Context Awareness**: AutoGen emphasizes context-based solutions, allowing agents to provide more relevant and accurate responses. This can lead to better decision-making and interactions.

5. **Scalability**: The architecture is designed to scale efficiently, making it suitable for handling varying workloads and complexities as your project grows.

6. **Collaboration**: It fosters collaboration among various agents, which can lead to more comprehensive insights and collaborative problem-solving.

These advantages make AutoGen a strong consideration for enhancing the capabilities and effectiveness of AI agents in your project. 

TERMINATE

## Cons of AutoGen:
Here are some cons associated with using AutoGen for an AI Agent project:

1. **Reliability Issues**: AutoGen may not always provide consistent performance, particularly in customer-facing applications where reliability is crucial.

2. **Complexity**: The lack of a visual builder or no-code editor translates to a steeper learning curve for non-technical users, which could hinder adoption and speed of development.

3. **Limited Parallelization**: AutoGen may struggle with tasks requiring quick processing times due to its inability to effectively parallelize tasks, potentially leading to slower response times.

4. **Dependency on Frameworks**: If extensive customization is needed, you might find yourself overly reliant on AutoGen and its underlying frameworks, which can increase complexity and restrict flexibility.

5. **Potential for Hallucination**: As with many LLMs, AutoGen may generate outputs that are inaccurate or nonsensical, which poses risks if not managed carefully.

These factors should be carefully considered when deciding whether to use AutoGen for your project. 

TERMINATE



## Decision:

Based on the analysis of the pros and cons presented by your research team, I recommend proceeding with AutoGen for your project. 

**Rationale**: The advantages of self-improvement, dynamic role-playing, efficiency, context awareness, scalability, and collaboration significantly outweigh the drawbacks. While reliability and complexity are valid concerns, the potential for enhanced operational efficiency and the ability to adapt and learn can provide substantial benefits, especially if managed with a robust oversight strategy. The overall capabilities of AutoGen align well with the goals of improving AI agent performance and optimizing workflows, making it a worthwhile investment.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()